In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import gc
import os
import datetime
from math import floor
from scipy.stats import poisson
from scipy.stats import binom


# Import Data

In [ ]:
soccerResults = pd.read_csv("results.csv")
soccerResults['date'] = pd.to_datetime(soccerResults['date'])

In [ ]:
soccerResults['year'] = soccerResults['date'].dt.year

In [ ]:
matches = soccerResults[soccerResults['date'] > '2018-01-01']

In [ ]:
matches.head()

# Clean Data

In [ ]:
to_drop = ['Italy','Turkey','Switzerland','Wales',
           'Belgium','Russia','Denmark','Finland',
           'Netherlands','Austria','Ukraine','Macedonia',
          'England','Croatia','Scotland','Czech Republic',
          'Spain','Sweden','Poland','Slovakia',
          'France','Portugal','Germany','Hungary']

In [ ]:
groupa = matches[matches.home_team.isin(to_drop)]
groupaAway = matches[matches.away_team.isin(to_drop)]

In [ ]:
groupa.head()

# Teams to Check

In [ ]:
# Type Teams Here
Team1Name = "Italy"
Team2Name = "Turkey"

# Functions


In [ ]:

def getAttack(Team):
    teamAverage = groupa[groupa['home_team'] == Team]
    teamAwayAverage = groupaAway[groupaAway['away_team'] == Team]  
    teamAttTotal = (teamAverage['home_score'].mean() + teamAwayAverage['away_score'].mean()) / 2
    return teamAttTotal

def getDef(Team):
    teamAverage = groupa[groupa['home_team'] == Team]
    teamAwayAverage = groupaAway[groupaAway['away_team'] == Team]
    teamDefTotal = (teamAverage['away_score'].mean() + teamAwayAverage['home_score'].mean()) / 2
    return teamDefTotal

def floored_percentage(val, digits):     
    val *= 10 ** (digits + 2)
    return '{1:.{0}f}%'.format(digits, floor(val) / 10 ** digits)

# Main Code

In [ ]:
#Global Averages from 2018
GroupAverage = matches['home_score'].mean()
Group2Average = matches['away_score'].mean()
GlobalAvg = (GroupAverage + Group2Average) / 2

#Get Teams Attack and Defense
Team1Att = getAttack(Team1Name) / GlobalAvg
Team1Def = getDef(Team1Name) / GlobalAvg
Team2Att = getAttack(Team2Name) / GlobalAvg
Team2Def = getDef(Team2Name) / GlobalAvg

print(GlobalAvg)
print(Team2Att)
print(Team2Def)
print(Team1Def)
print(Team1Att)

#Expected Goals for both teams Vs Each Other
#Mutiplty attack * opposition defense * global average
Team1ExG = Team1Att * Team2Def * GlobalAvg
Team2ExG = Team2Att * Team1Def * GlobalAvg

#Add into Data Frame with 5 goals the highest score
n = np.arange(0,6)
y2 = poisson.pmf(n,Team2ExG)
y = poisson.pmf(n,Team1ExG)
data = {Team1Name: y,
        Team2Name: y2}
  
df = pd.DataFrame(data)
cm = sns.light_palette("green", as_cmap=True)
s = df.style.background_gradient(cmap=cm)
df = (100 * df).round(2).astype(str) + '%'


###### Output

In [ ]:
df.T

###### Colour Scheme

In [ ]:
s